In [ ]:
# Importações
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

In [ ]:
# Definição de funções necessárias

def get_noticias(url, keywords):
    """
    Função que coleta notícias de uma página específica baseada em palavras-chave nos links.
    Também coleta a data de publicação, se disponível.
    """
    noticias = []
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.encoding = response.apparent_encoding  # Ajuste para decodificação correta
        soup = BeautifulSoup(response.content, "html.parser")

        # Encontrar todos os links na página
        links = soup.find_all('a', href=True)

        # Filtrar e armazenar apenas os links que contêm as palavras-chave
        for link_tag in links:
            link = link_tag['href']
            title = link_tag.get_text(strip=True)
            
            # Verificar se a palavra-chave está no link ou no título
            if any(keyword in title.lower() for keyword in keywords):
                # Se o link não é absoluto, fazê-lo absoluto
                if not link.startswith('http'):
                    link = urljoin(url, link)

                # Tentar capturar a data de publicação da notícia
                date = 'Data não disponível'
                try:
                    date_tag = soup.find('time')  # Se houver uma tag <time> na página
                    if date_tag and date_tag.has_attr('datetime'):
                        date = date_tag['datetime']
                    else:
                        # Tentativa alternativa, por exemplo, se a data estiver em um <span> com uma classe específica
                        date_tag = soup.find('span', class_='data-publicacao')
                        date = date_tag.get_text(strip=True) if date_tag else 'Data não disponível'
                except Exception:
                    pass

                # Adicionar a notícia à lista, incluindo a data
                if title:
                    noticias.append({"title": title, "link": link, "date": date})
    
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
    
    return noticias

def crawl_site(url, keywords, depth=3):
    """
    Função recursiva para percorrer todo o site, coletando notícias relevantes.
    Agora coleta a data da notícia.
    """
    visited_links = set()
    noticias = []

    def _crawl(url, current_depth):
        # Verificar se o link já foi visitado ou se a profundidade máxima foi atingida
        if current_depth == 0 or url in visited_links:
            return
        
        visited_links.add(url)  # Marcar o link como visitado
        noticias_atual = get_noticias(url, keywords)

        # Remover links duplicados antes de adicionar à lista de notícias
        for noticia in noticias_atual:
            if noticia["link"] not in {n["link"] for n in noticias}:
                noticias.append(noticia)

        try:
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            response.encoding = response.apparent_encoding
            soup = BeautifulSoup(response.content, "html.parser")

            # Encontrar todos os links na página e seguir
            links = soup.find_all('a', href=True)

            for link_tag in links:
                link = link_tag['href']
                if not link.startswith('http'):
                    link = urljoin(url, link)

                # Verificar se o link é interno e não foi visitado
                if url in link and link not in visited_links:
                    _crawl(link, current_depth - 1)

        except requests.exceptions.RequestException as e:
            print(f"Erro ao acessar {url}: {e}")

    _crawl(url, depth)
    return noticias

def get_text(url):
    '''
    Função 
    '''
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        response.encoding = response.apparent_encoding
        soup = BeautifulSoup(response.content, 'html.parser')

        for script in soup(['script', 'style']):
            script.extract()

        text = soup.get_text()

        lines = (line.strip() for line in text.splitlines())

        chunks = (phrase.strip() for line in lines for phrase in line.split())

        text = ' '.join(chunk for chunk in chunks if chunk)

        return text
    
    except Exception as e:
        print(f"Erro ao obter texto de {url}: {e}")
        return ""

def coluna_texto(df):
    if 'link' in df:
        df['corpo_texto'] = df['link'].apply(get_text)
    else:
        pass
    
def extrair_para_df(data_dict, keywords):
    base_url = data_dict['base_url']
    depth = data_dict['depth']
    
    # Coletar notícias do site inteiro, seguindo links internos
    df_nome = pd.DataFrame(crawl_site(base_url, keywords, depth=depth))
    
    # Remover títulos duplicados
    df_nome = df_nome.drop_duplicates(subset='title')
    
    # Adicionar a coluna 'full_text' com o texto completo extraído de cada link
    coluna_texto(df_nome)
    
    return df_nome

In [ ]:
# Palavras-chave a serem buscadas
keywords = ["vacina", "vacinação", "vacinacao", "covid", "pandemia", "corona", "coronavírus", "coronavirus", "quarentena", "sars-cov-2"]

In [ ]:
# Definição dos dados a serem extraídos
true_data = {
    'g1 saude':{
        'base_url':'https://g1.globo.com/saude/',
        'depth':4
    },
    'gov saude':{
        'base_url':'https://www.gov.br/saude/pt-br',
        'depth':3
    }
}

fake_data = {
    'fato ou fake g1':{
        'base_url':'https://g1.globo.com/fato-ou-fake/',
        'depth':6
    },
    'aos fatos':{
        'base_url':'https://www.aosfatos.org',
        'depth':3
    },
    'gov fake':{
        'base_url':'https://www.saude.gov.br/fakenews',
        'depth':3
    },
    'boatos':{
        'base_url':'https://www.boatos.org',
        'depth':3
    },
    'mpv':{
        'base_url':'https://medicospelavidacovid19.com.br/',
        'depth':5
    }    
}

In [ ]:
# Extrai dados da internet e cria uma lista com os dfs (falsos e verdadeiros)
fake_dfs = []
for data_dict in fake_data.values():
    fake_dfs.append(extrair_para_df(data_dict, keywords))
    
true_dfs = []
for data_dict in true_data.values():
    true_dfs.append(extrair_para_df(data_dict, keywords))

In [ ]:
# Devido à estrutura desse site, precisamos mudar a sintaxe para o www.e-farsas.com
base_url = "https://www.e-farsas.com/secoes/verdadeiro-2/page"

# Coletar notícias de todas as páginas do site, seguindo links internos
df_efarsa_verdadeiro = []
for i in range(1, 159):  # Número de páginas a ser percorrido
    page_url = f"{base_url}{i}"
    df_efarsa_verdadeiro.append(extrair_para_df({'base_url':page_url,'depth':3}, keywords))

df_efarsa_verdadeiro =  pd.concat(df_efarsa_verdadeiro, ignore_index=True)
true_dfs.append(df_efarsa_verdadeiro)


base_url = "https://www.e-farsas.com/secoes/falso-2/page"

df_efarsa_falso = []
for i in range(1, 159):
    page_url = f"{base_url}{i}"
    df_efarsa_falso.append(extrair_para_df({'base_url':page_url,'depth':3}, keywords))

df_efarsa_falso =  pd.concat(df_efarsa_falso, ignore_index=True)
fake_dfs.append(df_efarsa_falso)

In [ ]:
# Junta os dfs fakes e salva
df_combinedfake = pd.concat(fake_dfs, ignore_index=True)
df_combinedfake['classe'] = 0
df_combinedfake.to_csv("dados/df_combinedfake.csv", index=False, encoding='utf-8')
display(df_combinedfake)

# Junta os verdadeiros e salva
df_combinedtrue = pd.concat(true_dfs, ignore_index=True)
df_combinedfake['classe'] = 1
df_combinedtrue.to_csv("dados/df_combinedtrue.csv", index=False, encoding='utf-8')
display(df_combinedtrue)